In [3]:
from pathlib import Path
import json
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)

In [4]:
DATA_DIR       = Path("data")
NPZ_PATH       = DATA_DIR / "part4_Xy_arrays.npz"
FEATS_JSON     = DATA_DIR / "part4_feature_names.json"

BEST_MODEL_PATH   = DATA_DIR / "part5_best_model.pkl"
SUMMARY_CSV_PATH  = DATA_DIR / "part5_model_summary.csv"
TEST_PRED_PATH    = DATA_DIR / "part5_test_predictions.csv"
META_JSON_PATH    = DATA_DIR / "part5_best_model_meta.json"


In [5]:
assert NPZ_PATH.exists(), f"Missing {NPZ_PATH}"
assert FEATS_JSON.exists(), f"Missing {FEATS_JSON}"

arrs = np.load(NPZ_PATH)
X_train, y_train = arrs["X_train"], arrs["y_train"]
X_test,  y_test  = arrs["X_test"],  arrs["y_test"]

with open(FEATS_JSON, "r") as f:
    FEATS_META = json.load(f)
FEATURE_NAMES = FEATS_META["FEATURE_NAMES"]
LABEL_COL     = FEATS_META["LABEL_COL"]

In [6]:
print("Shapes:")
print("  X_train:", X_train.shape, "y_train:", y_train.shape)
print("  X_test :", X_test.shape,  "y_test :", y_test.shape)


Shapes:
  X_train: (588899, 29) y_train: (588899,)
  X_test : (147225, 29) y_test : (147225,)


In [14]:

# ---------- 5.1 (Optional) Training subsample for faster prototyping ----------
# Set TRAIN_SUBSAMPLE = None to use ALL rows; or set an integer (e.g., 200_000)
TRAIN_SUBSAMPLE = None  # e.g., 200_000

if (TRAIN_SUBSAMPLE is not None) and (TRAIN_SUBSAMPLE < len(y_train)):
    X_train_sub, _, y_train_sub, _ = train_test_split(
        X_train, y_train,
        train_size=TRAIN_SUBSAMPLE,
        random_state=42,
        stratify=y_train
    )
else:
    X_train_sub, y_train_sub = X_train, y_train

In [22]:
# Make a small validation split from training for early stopping & threshold tuning
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_sub, y_train_sub,
    test_size=0.10, random_state=42, stratify=y_train_sub
)


In [23]:
# ---------- 5.2 Define models ----------
# 1) RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=1,
    n_jobs=-1,
    random_state=42
)

In [24]:
# 2) XGBoost
xgb_ok = True
try:
    from xgboost import XGBClassifier
    xgb = XGBClassifier(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        tree_method="hist",
        random_state=42,
        n_jobs=-1,
        eval_metric="auc"
    )
except Exception as e:
    xgb_ok = False
    xgb_err = str(e)

In [25]:
# 3) LightGBM
lgbm_ok = True
try:
    from lightgbm import LGBMClassifier, early_stopping, log_evaluation
    lgbm = LGBMClassifier(
        n_estimators=800,
        learning_rate=0.05,
        num_leaves=31,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        random_state=42,
        n_jobs=-1
    )
except Exception as e:
    lgbm_ok = False
    lgbm_err = str(e)


In [26]:
# ---------- 5.3 Train each model (using train/val split) ----------
results = []
fitted = {}

In [27]:
# Helper to compute metrics
def compute_metrics(name, y_true, proba, thr=0.5):
    pred = (proba >= thr).astype(int)
    return {
        "model": name,
        "roc_auc": float(roc_auc_score(y_true, proba)),
        "pr_auc": float(average_precision_score(y_true, proba)),
        "accuracy@0.5": float(accuracy_score(y_true, pred)),
        "precision@0.5": float(precision_score(y_true, pred, zero_division=0)),
        "recall@0.5": float(recall_score(y_true, pred, zero_division=0)),
        "f1@0.5": float(f1_score(y_true, pred, zero_division=0)),
    }

In [28]:
def find_best_f1_threshold(y_true, proba, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 19)
    scores = [(t, f1_score(y_true, (proba >= t).astype(int), zero_division=0)) for t in grid]
    best_t, best_f1 = max(scores, key=lambda x: x[1])
    return float(best_t), float(best_f1)


In [29]:
# --- RandomForest ---
rf.fit(X_tr, y_tr)
rf_val_proba = rf.predict_proba(X_val)[:, 1]
rf_metrics_val = compute_metrics("RandomForest (val)", y_val, rf_val_proba)
results.append(rf_metrics_val)
fitted["RandomForest"] = rf

In [ ]:
# --- XGBoost ---
if xgb_ok:
    xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False,
        early_stopping_rounds=50
    )
    xgb_val_proba = xgb.predict_proba(X_val)[:, 1]
    xgb_metrics_val = compute_metrics("XGBoost (val)", y_val, xgb_val_proba)
    results.append(xgb_metrics_val)
    fitted["XGBoost"] = xgb
else:
    print("XGBoost not available:", xgb_err)


In [30]:
# --- LightGBM ---
if lgbm_ok:
    lgbm.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(50, verbose=False)]
    )
    lgbm_val_proba = lgbm.predict_proba(X_val)[:, 1]
    lgbm_metrics_val = compute_metrics("LightGBM (val)", y_val, lgbm_val_proba)
    results.append(lgbm_metrics_val)
    fitted["LightGBM"] = lgbm
else:
    print("LightGBM not available:", lgbm_err)


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 115925, number of negative: 414084
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 948
[LightGBM] [Info] Number of data points in the train set: 530009, number of used features: 29
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Curr

In [31]:
summary_val = pd.DataFrame(results).sort_values("roc_auc", ascending=False).reset_index(drop=True)
print("\n=== Validation summary (higher ROC-AUC is better) ===")
display(summary_val)



=== Validation summary (higher ROC-AUC is better) ===


,model,roc_auc,pr_auc,accuracy@0.5,precision@0.5,recall@0.5,f1@0.5
0,LightGBM (val),0.660899,0.345702,0.781678,0.532432,0.015294,0.029734
1,RandomForest (val),0.587400,0.273594,0.732790,0.305173,0.173589,0.221298


In [32]:
# ---------- 5.4 Pick best by validation ROC-AUC & refit on FULL TRAIN ----------
# Choose best name among keys present in `fitted`
def _name_from_row(row_model):
    # row_model like "RandomForest (val)" -> "RandomForest"
    return row_model.split(" ", 1)[0]

best_row = summary_val.iloc[0]
BEST_MODEL_NAME = _name_from_row(best_row["model"])
print("\nBest (val):", BEST_MODEL_NAME)



Best (val): LightGBM


In [33]:
# Refit best model on ALL of X_train (with 10% as internal val if boosting)
if BEST_MODEL_NAME == "RandomForest":
    best_model = RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_leaf=1,
        n_jobs=-1, random_state=42
    ).fit(X_train, y_train)

elif BEST_MODEL_NAME == "XGBoost":
    best_model = XGBClassifier(
        n_estimators=600, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
        tree_method="hist", random_state=42, n_jobs=-1, eval_metric="auc"
    )
    X_tr_full, X_val_full, y_tr_full, y_val_full = train_test_split(
        X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
    )
    best_model.fit(
        X_tr_full, y_tr_full,
        eval_set=[(X_val_full, y_val_full)],
        verbose=False,
        early_stopping_rounds=50
    )

elif BEST_MODEL_NAME == "LightGBM":
    best_model = LGBMClassifier(
        n_estimators=800, learning_rate=0.05, num_leaves=31,
        feature_fraction=0.8, bagging_fraction=0.8,
        random_state=42, n_jobs=-1
    )
    X_tr_full, X_val_full, y_tr_full, y_val_full = train_test_split(
        X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
    )
    best_model.fit(
        X_tr_full, y_tr_full,
        eval_set=[(X_val_full, y_val_full)],
        callbacks=[early_stopping(50, verbose=False)]
    )

else:
    raise RuntimeError("Unknown BEST_MODEL_NAME.")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 115925, number of negative: 414084
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 948
[LightGBM] [Info] Number of data points in the train set: 530009, number of used features: 29
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Curr

In [34]:
# ---------- 5.5 Evaluate on TEST ----------
proba_test = best_model.predict_proba(X_test)[:, 1]

# Default-threshold metrics
metrics_test = compute_metrics(BEST_MODEL_NAME + " (test)", y_test, proba_test, thr=0.5)


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


In [35]:
# F1-tuned threshold (on training probabilities)
proba_train = best_model.predict_proba(X_train)[:, 1]
best_t, best_f1_train = find_best_f1_threshold(y_train, proba_train)
pred_test_tuned = (proba_test >= best_t).astype(int)

metrics_test_tuned = {
    "model": BEST_MODEL_NAME + " (test, tuned)",
    "threshold*": best_t,
    "accuracy@t": float(accuracy_score(y_test, pred_test_tuned)),
    "precision@t": float(precision_score(y_test, pred_test_tuned, zero_division=0)),
    "recall@t": float(recall_score(y_test, pred_test_tuned, zero_division=0)),
    "f1@t": float(f1_score(y_test, pred_test_tuned, zero_division=0)),
    "roc_auc": float(roc_auc_score(y_test, proba_test)),
    "pr_auc": float(average_precision_score(y_test, proba_test))
}


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


In [36]:
print("\n=== Test metrics (threshold=0.5) ===")
print(metrics_test)
print("\n=== Test metrics (tuned threshold) ===")
print(metrics_test_tuned)



=== Test metrics (threshold=0.5) ===
{'model': 'LightGBM (test)', 'roc_auc': 0.6396863299011186, 'pr_auc': 0.3259998686288026, 'accuracy@0.5': 0.7812124299541519, 'precision@0.5': 0.49476987447698745, 'recall@0.5': 0.014688984814136207, 'f1@0.5': 0.02853092861235938}

=== Test metrics (tuned threshold) ===
{'model': 'LightGBM (test, tuned)', 'threshold*': 0.2, 'accuracy@t': 0.5533503141450161, 'precision@t': 0.28230943885825494, 'recall@t': 0.6757243563864477, 'f1@t': 0.3982393206193492, 'roc_auc': 0.6396863299011186, 'pr_auc': 0.3259998686288026}


In [37]:
# ---------- 5.6 Save artifacts ----------
# 1) Best model
joblib.dump(best_model, BEST_MODEL_PATH)


['data/part5_best_model.pkl']

In [38]:
# 2) Model comparison summary (validation metrics + test metrics for best)
summary_val.to_csv(SUMMARY_CSV_PATH, index=False)


In [39]:
# 3) Test predictions CSV (for downstream dashboards or error analysis)
test_pred_df = pd.DataFrame({
    "y_test": y_test,
    "p_delay_best": proba_test,
    "pred_0_5": (proba_test >= 0.5).astype(int),
    "pred_tuned": pred_test_tuned
})
test_pred_df.to_csv(TEST_PRED_PATH, index=False)


In [40]:
# 4) Metadata about the run
META = {
    "best_model_name": BEST_MODEL_NAME,
    "feature_names": FEATURE_NAMES,
    "threshold_tuned": best_t,
    "train_subsample": TRAIN_SUBSAMPLE,
    "paths": {
        "best_model_path": str(BEST_MODEL_PATH),
        "summary_csv": str(SUMMARY_CSV_PATH),
        "test_predictions": str(TEST_PRED_PATH)
    },
    "test_metrics@0.5": metrics_test,
    "test_metrics@tuned": metrics_test_tuned
}
with open(META_JSON_PATH, "w") as f:
    json.dump(META, f, indent=2)


In [41]:
print("\nSaved artifacts:")
print("  Best model:", BEST_MODEL_PATH)
print("  Comparison:", SUMMARY_CSV_PATH)
print("  Test preds:", TEST_PRED_PATH)
print("  Meta JSON :", META_JSON_PATH)


Saved artifacts:
  Best model: data/part5_best_model.pkl
  Comparison: data/part5_model_summary.csv
  Test preds: data/part5_test_predictions.csv
  Meta JSON : data/part5_best_model_meta.json
